In [9]:
import pygame
import numpy as np
import random
import os
import math
from screeninfo import get_monitors
import time
import pandas as pd

OUTPUT_PATH = r"C:\Users\yonir\Desktop\smooth tracking\data\third Batch"
FIELD_OF_VIEW = 90.0
CHECKER_SIZE = 10.0 # Deg
BAR_WIDTH = 2 #Number of squares
BOUNCING_LIMITS = 45.0  # Deg
SIDE_DURATION = 1.0 # Seconds
SIDES_PER_PHASE = 4 #How many side motions should an object during the stimulus phase 
BREAK_DURATION = 1.0 # Seconds
ITERATION = 20 # Number of loops
TARGET_FPS = 60

# Initialize Pygame
monitors = get_monitors()
monitor = monitors[-1]
if len(monitors) > 1:
    os.environ['SDL_VIDEO_WINDOW_POS'] = "%d,%d" % (monitors[0].width, 0) #this will push the pygame window in the second monitor
    
PIX_WIDTH = monitor.width
PIX_HEIGHT = monitor.height
PIX2DEG = PIX_WIDTH / FIELD_OF_VIEW
DEG_WIDTH = PIX_WIDTH / PIX2DEG
DEG_HEIGHT = PIX_HEIGHT / PIX2DEG


BAR_WIDTH = BAR_WIDTH * CHECKER_SIZE
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

def generate_checkerboard(width, height):
    squares_per_row, squares_per_col = int(width / CHECKER_SIZE), int(height / CHECKER_SIZE)
    checkerboard = np.zeros((squares_per_row, squares_per_col), dtype=int)
    for i in range(squares_per_row):
        for j in range(squares_per_col):
            checkerboard[i, j] = random.choice([0, 1])  # 0 for black, 1 for white
    return checkerboard

def draw_rectangle(color, x, y, width, height):
    pygame.draw.rect(screen, color, (x*PIX2DEG, y*PIX2DEG, width*PIX2DEG, height*PIX2DEG))

def draw_checkerboard(pattern, offset_x, offset_y):
    for i in range(pattern.shape[0]):
        for j in range(pattern.shape[1]):
            color = WHITE if pattern[i, j] == 1 else BLACK
            draw_rectangle(color, (i * CHECKER_SIZE) + offset_x, (j * CHECKER_SIZE) + offset_y, CHECKER_SIZE, CHECKER_SIZE)

#design stimuli (x is themoving object horizontal postion) 
PHASES = [
    lambda x: (
        screen.fill(WHITE),
        draw_rectangle(BLACK, x, 0, BAR_WIDTH, DEG_HEIGHT)
    ),
    lambda x: (
        draw_checkerboard(checkerboard, 0, 0),
        draw_checkerboard(bar, x, 0)
    ),
    lambda x: (
        draw_checkerboard(checkerboard, 0, 0),
        draw_rectangle(BLACK, x, 0, BAR_WIDTH, DEG_HEIGHT)
    ),
    lambda x: (
        draw_checkerboard(checkerboard, x, 0)
    )
]

PHASE_OBECTS = [BAR_WIDTH, BAR_WIDTH, BAR_WIDTH, DEG_WIDTH*2]
PHASE_NUMBER = len(PHASES)

pygame.init()
screen = pygame.display.set_mode((PIX_WIDTH, PIX_HEIGHT))
clock = pygame.time.Clock()

now = time.time()
Abs_side_start = 0.0
Abs_phase_start = 0.0
Abs_loop_start = 0.0
current_phase = 0
current_side = 0
current_loop = 0
new_phase = True
new_side = True
new_loop = True
in_break = True
log = []
frames_count = 0
time_start = time.time()
checkerboard = generate_checkerboard(DEG_WIDTH*2, DEG_HEIGHT+CHECKER_SIZE)
bar = generate_checkerboard(BAR_WIDTH, DEG_HEIGHT+CHECKER_SIZE)

while current_loop <= ITERATION:

    quit = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            quit = True
    if quit:
        break
    
    theoretical_elapsed_time = frames_count*(1/TARGET_FPS)
    dt = time.time() - time_start
    
    if (dt > theoretical_elapsed_time):    
        frames_count += 1
        now = time.time()

    if new_loop:
        Abs_loop_start = now
        current_phase = 0
        new_phase = True
        new_loop = False
    
    if new_phase:
        Abs_phase_start = now
        checkerboard = generate_checkerboard(DEG_WIDTH*2, DEG_HEIGHT+CHECKER_SIZE)
        bar = generate_checkerboard(BAR_WIDTH, DEG_HEIGHT+CHECKER_SIZE)
        current_side = 0
        new_phase = False
        
    phase_elapsed_time = now - Abs_phase_start - BREAK_DURATION
    loop_elapsed_time = now - Abs_loop_start
    
    if phase_elapsed_time < 0:
        in_break = True
        screen.fill(BLACK)
    
    else:
        if in_break:
            new_side = True
            in_break = False
        
        if new_side:
            Abs_side_start = now
            new_side = False
        
        side_elpased_time = now - Abs_side_start
            
        if side_elpased_time >= SIDE_DURATION:
            log.append(("side", current_side, Abs_side_start, now, now-Abs_side_start))
            new_side = True
            current_side += 1
        
        if current_side == SIDES_PER_PHASE:
            log.append(("phase", current_phase, Abs_phase_start, now, now-Abs_phase_start))
            new_phase = True
            current_phase += 1
        
        if current_phase >= PHASE_NUMBER:
            log.append(("loop", current_loop, Abs_loop_start, now, now-Abs_loop_start ))
            new_loop = True
            current_loop += 1
            
        if (not new_phase) & (not new_side) & (not new_loop):
            is_side_even = - 1 + (current_side%2 != 0)*2
            starting_pos = (DEG_WIDTH + (is_side_even*BOUNCING_LIMITS) - PHASE_OBECTS[current_phase])/2
            object_pos = starting_pos - (1-(SIDE_DURATION - side_elpased_time)/SIDE_DURATION)*BOUNCING_LIMITS*is_side_even
            PHASES[current_phase](object_pos)
    
    pygame.display.update()
    clock.tick(TARGET_FPS)

df = pd.DataFrame(log, columns=["Event", "ID", "AbsoluteStart", "AbsoluteEnd", "Duration"])
df.to_csv(os.path.join(OUTPUT_PATH, 'stimulus_log.csv'), index=False)
pygame.quit()